<a href="https://colab.research.google.com/github/Potdooshami/planar-domain-wall-network/blob/main/Homology_Z2Z2Z2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install hsnf

In [ ]:

from hsnf import column_style_hermite_normal_form, row_style_hermite_normal_form, smith_normal_form
#import
import numpy as np
import pandas as pd
import itertools

#my own class
class FiniteAbelianGroup:
  def __init__(self, value, modulus):
    self.modulus = np.array(modulus)
    self.value = np.mod(np.array(value),self.modulus)


  def __rmul__(self, Z):
    #return FiniteAbelianGroup(self.value + other.value, self.modulus)
    return FiniteAbelianGroup(self.value * Z, self.modulus)
  def __add__(self, other):
    return FiniteAbelianGroup(self.value + other.value, self.modulus)
  def __neg__(self):
    return FiniteAbelianGroup(-self.value, self.modulus)
  #def __rmul__(self,Z):


  #def __pow__(self, power):
    #self._check_modulus(other)
    #return FiniteAbelianGroup(self.value * power, self.modulus)

  def __str__(self):
    return f"z{self.value}"

  def __repr__(self):
    return f"z{self.value}"

  def __lt__(self, other):
    self._check_modulus(other)
    return tuple(self.value) < tuple(other.value)

  def __eq__(self, other):
    self._check_modulus(other)
    return tuple(self.value) == tuple(other.value)

  def __hash__(self):
    return hash((tuple(self.value), tuple(self.modulus)))
  def _check_modulus(self, other):
    """Helper method to check modulus compatibility."""
    if not isinstance(other, FiniteAbelianGroup) or not np.array_equal(self.modulus, other.modulus):
      raise ValueError("Cannot perform operation between groups of different moduli.")
  def generate_combinations(self):#helper
    input_array = self.modulus
    # Use itertools.product to generate Cartesian product
    combinations = list(itertools.product(*[range(n) for n in input_array]))
    # Convert the list of tuples into a 2D NumPy array
    return np.array(combinations)
  def all(self):
    combs = self.generate_combinations()
    tmp = []
    for ind in range(combs.shape[0]):
      tmp.append(FiniteAbelianGroup(combs[ind],self.modulus))
    return np.array(tmp)
  def table(self):
    allel = self.all()
    foo = np.matmul(allel.reshape(-1,1),allel.reshape(1,-1))
    #tbl = pd.DataFrame(foo,index=allel,columns=allel)
    strarr =  allel.astype(str)
    tbl = pd.DataFrame(foo,index=strarr,columns=strarr)
    return tbl
  def order(self):
    return self.modulus.prod()
class Z2Z2(FiniteAbelianGroup):
  def __init__(self, value=[0,0]):
    super().__init__(value, [2,2])
class Z222(FiniteAbelianGroup):
  def __init__(self, value=[0,0]):
    super().__init__(value, [2,2,2])
class Z33(FiniteAbelianGroup):
  def __init__(self, value=[0,0]):
    super().__init__(value, [3,3])
#my own function
def get_powerset(s):
  result = []
  for r in range(len(s)+1):
    for combo in itertools.combinations(s, r):
      result.append(''.join(combo))  # Join the tuple into a string
  return result
def sortAlp(str):
  return ''.join(sorted(str))
def docl2lmap(dcol):
  #doubleColumn to linearMap
  dcol = dcol.set_index(dcol.columns[0])
  dcol = dcol.astype(str)
  lmap = pd.get_dummies(dcol,prefix='', prefix_sep='').astype(int)
  return lmap
def combdpi(pil_,bndf_):
  arr = np.matmul(pil_.values.T,bndf_.values)
  return pd.DataFrame(arr,columns=bndf_.columns,index = pil_.columns)
def is_cyclic_permutation(A, B):
  A = list(A)
  B = list(B)
  if len(A) != len(B):
    return False
  AA = A + A
  for i in range(len(A)):
    if AA[i: i + len(A)] == B:
      return True
  return False
def sortvort(vort):
  svort = sorted(vort)
  svort = np.array(svort)
  is_cycperm = is_cyclic_permutation(vort, svort)
  if not(is_cycperm):
    svort = svort[[0,2,1]]
  return svort
def anti_op(vort):
  antivort = -vort[[2,1,0]]
  return antivort
def chir_op(vort):
  chirvort = vort[[0,2,1]]
  return chirvort
def anti_unique(vort):
  vort = sortvort(vort)
  avort = sortvort(anti_op(vort))
  #print('avort',avort)
  is_vort_front =  tuple(vort)<tuple(avort)
  #print('au',is_vort_front)
  if is_vort_front:
    vortUq = vort
  else:
    vortUq = avort
  vort_au = [vortUq,is_vort_front*2-1]
  return vort_au

#Homology function
def kernel(A, tol=1e-5):
    """
    Return a matrix whose column space is the kernel of A.
    The tol parameter is the threshold below which a singular value is considered to be zero.
    Taken from: https://github.com/kb1dds/simplicialHomology/blob/master/simplicialHomology.py
    """
    _, s, vh = np.linalg.svd(A)
    singular = np.zeros(vh.shape[0], dtype=float)
    singular[:s.size] = s
    null_space = np.compress(singular <= tol, vh, axis=0)
    return null_space.T
def cokernel(A, tol=1e-5):
    """
    Return a matrix whose column space is the cokernel of A.
    The tol parameter is the threshold below which a singular value is considered to be zero.
    Taken from: https://github.com/kb1dds/simplicialHomology/blob/master/simplicialHomology.py
    """
    u, s, _ = np.linalg.svd(A)
    singular = np.zeros(u.shape[1], dtype=float)
    singular[:s.size] = s
    return np.compress(singular <= tol, u, axis=1)
def get_faces(lst):
    """Compute all the possible faces by iteratively deleting vertices"""
    return [lst[:i] + lst[i+1:] for i in range(len(lst))]
def get_coeff(simplex, faces):
    """
    If simplex is not in the list of faces, return 0.
    If it is, return index parity.
    """
    if simplex in faces:
        idx = faces.index(simplex)
        return 1 if idx%2==0 else -1
    else:
        return 0
def boundary(complex):
    """
    Given an abstract simplicial complex specified as a list of lists of vertices, return a
    list of boundary operators in matrix form.
    """
    # Get maximal simplex dimension
    ##maxdim = len(max(complex, key=len))
    # Group simplices by (ascending) dimension and sort them lexicographically
    ##simplices = [sorted([spx for spx in complex if len(spx)==i]) for i in range(1,maxdim+1)]
    simplices = chainSort(complex)
    # Iterate over consecutive groups (dim k and k+1)
    bnd = []
    for spx_k, spx_kp1 in zip(simplices, simplices[1:]):
        mtx = []
        for sigma in spx_kp1:
            faces = get_faces(sigma)
            mtx.append([get_coeff(spx, faces) for spx in spx_k])
        bnd.append(np.array(mtx).T)

    return bnd
def chainSort(complex):
  # Get maximal simplex dimension
    maxdim = len(max(complex, key=len))
    # Group simplices by (ascending) dimension and sort them lexicographically
    simplices = [sorted([spx for spx in complex if len(spx)==i]) for i in range(1,maxdim+1)]
    return simplices
def homology(boundary_ops, tol=1e-5):
    """
    Given a list of boundary operators, return a list of matrices whose columns
    span the homology spaces.
    """
    # Insert zero maps
    mm = boundary_ops[-1].shape[1]
    nn = boundary_ops[0].shape[0]

    boundary_ops.insert(0, np.ones(shape=(0, nn)))
    boundary_ops.append(np.ones(shape=(mm, 0)))

    H = []
    for del_k, del_kp1 in zip(boundary_ops, boundary_ops[1:]):
        # Compute a basis for the kernel of the next map
        kappa = kernel(del_k, tol)
        # The chain complex induces a map m from previous space to the kernel of next map
        # Solve d_{k} = kappa \circ m for m
        psi, _, _, _ = np.linalg.lstsq(kappa, del_kp1, rcond=None)
        # The cokernel of m is precisely those elements of the kernel of the next map
        # that are not in the image of m (or d_k for that matter), that's homology
        ksi = cokernel(psi, tol)
        # Express a basis for the homology thought of as a subspace of C_k
        # using composition of maps
        H.append(np.dot(kappa, ksi))

    return H
def betti(H):
  """Compute the dimensions of each homology space output by the homology() function"""
  return [basis.shape[1] for basis in H]

In [ ]:
#G = Z2Z2([1,1]).all()
G = FiniteAbelianGroup([0,0],[3,3]).all()
vertexNames = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ가나다라마바사아자차카타파하'
X = get_powerset(vertexNames[:len(G)])
bnd = boundary(X)#list of dk
bndf = []
simps = chainSort(X)#list of Ck
for k in  range(len(bnd)):
  bndf.append(pd.DataFrame(bnd[k],index=simps[k],columns=simps[k+1]))
bndf.insert(0,0)
bnd = bndf



In [ ]:
#G: finite Abelian Group
Cn = simps # |G| complex
Cn_ =['C0_','C1_','C2_','C3_']
#bnd #Cn->Cn-1
pi=['pi0','pi1','pi2','pi3'] #Cn->Cn_
pil =['pil0','pil1','pil2','pil3'] #Cn->Cn_
dpi = [0,'dpi1','dpi2','dpi3'] #Cn->Cn-1_


In [ ]:



n = 0
pi_tmp = pd.DataFrame(G,columns=["G+"])
pi_tmp['C0'] = simps[n]
pi_tmp = pi_tmp[['C0','G+']]
pi[n] = pi_tmp

pil[n] = docl2lmap(pi[n])


#print(pil[n])
#print(bnd[n+1])
dpi_tmp = combdpi(pil[n],bnd[n+1])
dpi[n+1] = dpi_tmp
#print(dpi[n+1])

C1_  = np.matmul(dpi[n+1].values.T,G[0].all().reshape(-1,1))
#print(C1_)

바운더리 함수 데이터프레임화

In [ ]:
n = 1
pi_tmp = pd.DataFrame(simps[n],columns = ['C1'])
pi_tmp['C1_'] = C1_.reshape(-1)
pi_tmp
pi[n] = pi_tmp

pil[n] = docl2lmap(pi[n])
pil[n]
#print(pil[n])
#print(bnd[n+1])
dpi_tmp = combdpi(pil[n],bnd[n+1])
dpi[n+1] = dpi_tmp
#print(dpi[n+1])

In [ ]:
n = 2
pi_tmp = pd.DataFrame(simps[n],columns = ['C1'])
pi_tmp
pi[n] = pi_tmp
#print(pi[n])

In [ ]:
rvor = []
sgnAnti = []
n=2
#print(simps[n])
for ind in range(len(simps[n])):
  face = simps[n][ind]
  #print(face)
  edges = get_faces(face)
  #print(edges)
  char = pi[1].set_index('C1').loc[edges].values.reshape(-1)
  egSgn = bnd[2][face].loc[edges].values.reshape(-1)
  char = egSgn * char
  #print(char)
  auform = anti_unique(char)
  #print(auform)
  rvor.append(auform[0])
  sgnAnti.append(auform[1])


pi[2]['rvor'] = rvor
pi[2]['sgnAnti'] = sgnAnti
print(pi[2])



#print(anti_unique(char))

     C1                      rvor  sgnAnti
0   abc  [z[0 1], z[0 1], z[0 1]]        1
1   abd  [z[0 1], z[1 2], z[2 0]]        1
2   abe  [z[0 1], z[1 0], z[2 2]]        1
3   abf  [z[0 1], z[1 1], z[2 1]]        1
4   abg  [z[0 1], z[2 2], z[1 0]]        1
..  ...                       ...      ...
79  ehi  [z[0 1], z[2 2], z[1 0]]        1
80  fgh  [z[0 1], z[2 1], z[1 1]]        1
81  fgi  [z[0 1], z[2 2], z[1 0]]       -1
82  fhi  [z[0 1], z[2 0], z[1 2]]        1
83  ghi  [z[0 1], z[0 1], z[0 1]]        1

[84 rows x 3 columns]


In [ ]:
pitup = pi[2]['rvor'].apply(tuple)
tf = pd.get_dummies(pitup)
tfval = tf.values
sgnVal = pi[2]['sgnAnti'].values
sgnVal = np.tile(sgnVal.reshape(-1,1),(1,tfval.shape[1]))
dpi[3] = tfval*sgnVal
pil[2] = pd.DataFrame(dpi[3],columns = tf.columns,index = pi[2].set_index('C1').index)
pil[2] = pil[2].T
print(pil[2])
print(bnd[3])
foo = np.matmul(pil[2],bnd[3])
foo

C1                        abc  abd  abe  abf  abg  abh  abi  acd  ace  acf  \
(z[0 1], z[0 1], z[0 1])    1    0    0    0    0    0    0    0    0    0   
(z[0 1], z[1 0], z[2 2])    0    0    1    0    0    0    0   -1    0    0   
(z[0 1], z[1 1], z[2 1])    0    0    0    1    0    0    0    0   -1    0   
(z[0 1], z[1 2], z[2 0])    0    1    0    0    0    0    0    0    0   -1   
(z[0 1], z[2 0], z[1 2])    0    0    0    0    0    1    0    0    0    0   
(z[0 1], z[2 1], z[1 1])    0    0    0    0    0    0    1    0    0    0   
(z[0 1], z[2 2], z[1 0])    0    0    0    0    1    0    0    0    0    0   
(z[1 0], z[1 0], z[1 0])    0    0    0    0    0    0    0    0    0    0   
(z[1 0], z[1 1], z[1 2])    0    0    0    0    0    0    0    0    0    0   
(z[1 0], z[1 2], z[1 1])    0    0    0    0    0    0    0    0    0    0   
(z[1 1], z[1 1], z[1 1])    0    0    0    0    0    0    0    0    0    0   
(z[1 2], z[1 2], z[1 2])    0    0    0    0    0    0    0    0

,abcd,abce,abcf,abcg,abch,abci,abde,abdf,abdg,abdh,...,dehi,dfgh,dfgi,dfhi,dghi,efgh,efgi,efhi,eghi,fghi
"(z[0 1], z[0 1], z[0 1])",-1,-1,-1,-1,-1,-1,0,0,0,0,...,0,0,0,0,1,0,0,0,1,1
"(z[0 1], z[1 0], z[2 2])",1,1,1,0,0,0,1,0,0,0,...,-1,1,1,0,0,0,1,1,0,0
"(z[0 1], z[1 1], z[2 1])",1,1,1,0,0,0,0,1,0,0,...,1,-1,0,1,0,-1,-1,0,0,0
"(z[0 1], z[1 2], z[2 0])",1,1,1,0,0,0,-1,-1,-1,-1,...,0,0,-1,-1,0,1,0,-1,0,0
"(z[0 1], z[2 0], z[1 2])",0,0,0,1,1,1,1,1,0,1,...,0,0,1,1,-1,-1,0,1,-1,-1
"(z[0 1], z[2 1], z[1 1])",0,0,0,1,1,1,0,-1,0,0,...,-1,1,0,-1,-1,1,1,0,-1,-1
"(z[0 1], z[2 2], z[1 0])",0,0,0,1,1,1,-1,0,1,0,...,1,-1,-1,0,-1,0,-1,-1,-1,-1
"(z[1 0], z[1 0], z[1 0])",0,0,0,0,0,0,0,0,-1,0,...,0,0,0,0,0,0,0,0,0,0
"(z[1 0], z[1 1], z[1 2])",0,0,0,0,0,0,0,0,1,-1,...,0,0,0,0,0,0,0,0,0,0
"(z[1 0], z[1 2], z[1 1])",0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
foo1 =  foo.T
colinfo = foo1.columns
print(colinfo)
print(foo1)
arr = np.array([[3, 2, 2], [1, 3, 2]])
arr_sorted = np.sort(arr, axis=1)
foo1.sort_values(by = list(colinfo))

Index([(z[0 1], z[0 1], z[0 1]), (z[0 1], z[1 0], z[2 2]),
       (z[0 1], z[1 1], z[2 1]), (z[0 1], z[1 2], z[2 0]),
       (z[0 1], z[2 0], z[1 2]), (z[0 1], z[2 1], z[1 1]),
       (z[0 1], z[2 2], z[1 0]), (z[1 0], z[1 0], z[1 0]),
       (z[1 0], z[1 1], z[1 2]), (z[1 0], z[1 2], z[1 1]),
       (z[1 1], z[1 1], z[1 1]), (z[1 2], z[1 2], z[1 2])],
      dtype='object')
      (z[0 1], z[0 1], z[0 1])  (z[0 1], z[1 0], z[2 2])  \
abcd                        -1                         1   
abce                        -1                         1   
abcf                        -1                         1   
abcg                        -1                         0   
abch                        -1                         0   
...                        ...                       ...   
efgh                         0                         0   
efgi                         0                         1   
efhi                         0                         1   
eghi                   

,"(z[0 1], z[0 1], z[0 1])","(z[0 1], z[1 0], z[2 2])","(z[0 1], z[1 1], z[2 1])","(z[0 1], z[1 2], z[2 0])","(z[0 1], z[2 0], z[1 2])","(z[0 1], z[2 1], z[1 1])","(z[0 1], z[2 2], z[1 0])","(z[1 0], z[1 0], z[1 0])","(z[1 0], z[1 1], z[1 2])","(z[1 0], z[1 2], z[1 1])","(z[1 1], z[1 1], z[1 1])","(z[1 2], z[1 2], z[1 2])"
abcg,-1,0,0,0,1,1,1,0,0,0,0,0
abch,-1,0,0,0,1,1,1,0,0,0,0,0
abci,-1,0,0,0,1,1,1,0,0,0,0,0
abcd,-1,1,1,1,0,0,0,0,0,0,0,0
abce,-1,1,1,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
bdef,1,0,0,0,-1,-1,-1,0,0,0,0,0
cdef,1,0,0,0,-1,-1,-1,0,0,0,0,0
dghi,1,0,0,0,-1,-1,-1,0,0,0,0,0
eghi,1,0,0,0,-1,-1,-1,0,0,0,0,0


In [ ]:
summary = foo1.apply(tuple,axis=1)
q = summary.unique()
q.shape
q_ = np.array([list(t) for t in q])
print(q)
#print(q_)
q__ = [q_[:, i] for i in range(q_.shape[1])]
#print(q__)
#print(len(q))

[(-1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0)
 (-1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0)
 (0, 1, 0, -1, 1, 0, -1, 0, 0, 0, 0, 0)
 (0, 0, 1, -1, 1, -1, 0, 0, 0, 0, 0, 0)
 (0, 0, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0)
 (0, 0, 0, -1, 1, 0, 0, 0, -1, 1, 0, 0)
 (0, 0, 0, -1, 0, 1, 0, 0, 0, -1, 0, 1)
 (0, -1, 1, 0, 0, -1, 1, 0, 0, 0, 0, 0)
 (0, -1, 0, 0, 0, 0, 1, 0, -1, 1, 0, 0)
 (0, -1, 0, 0, 1, 0, 0, 1, 0, -1, 0, 0)
 (0, -1, 0, 0, 0, 1, 0, 0, 1, 0, -1, 0)
 (0, 0, -1, 0, 0, 0, 1, 0, 0, -1, 1, 0)
 (0, 0, -1, 0, 1, 0, 0, 0, 1, 0, 0, -1)
 (0, 0, -1, 0, 0, 1, 0, 0, -1, 1, 0, 0)
 (0, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)
 (0, 0, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0)
 (0, 1, 0, 0, -1, 0, 0, -1, 0, 1, 0, 0)
 (0, 1, 0, 0, 0, -1, 0, 0, -1, 0, 1, 0)
 (0, 1, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0)
 (0, 0, 1, 0, -1, 0, 0, 0, -1, 0, 0, 1)
 (0, 0, 1, 0, 0, -1, 0, 0, 1, -1, 0, 0)
 (0, 0, 1, 0, 0, 0, -1, 0, 0, 1, -1, 0)
 (0, 0, 0, 1, -1, 0, 0, 0, 1, -1, 0, 0)
 (0, 0, 0, 1, 0, -1, 0, 0, 0, 1, 0, -1)
 (0, 0, 0, 1, 0, 0, -1, 1, -1, 0, 0, 0)
 (

In [ ]:
def remove_null(A):
  Aarr =  np.array([list(t) for t in A])
  isnullrow = np.all(Aarr==0,axis=1)
  Anontrivial = A[~isnullrow]
  Atrivial = A[isnullrow]
  Asplit = (Anontrivial,Atrivial)
  return Asplit

def equivalence_info(A,iseq):
  Asplit = remove_null(A)
  Anontrivial = Asplit[0]
  Atrivial = Asplit[1]
  classes = equivalence_classes(Anontrivial, iseq)
  classes.append(set(Atrivial))
  return classes

def equivalence_classes(A, iseq):
  """
  Partitions a set into equivalence classes based on an equivalence relation.

  Args:
    A: The set as a list.
    iseq: A function that takes two elements from A and returns True if they are equivalent, False otherwise.

  Returns:
    A list of equivalence classes, each represented as a set.
  """
  classes = []
  remaining = set(A)
  while remaining:
    element = remaining.pop()
    new_class = {element}
    for other in remaining.copy():
      if iseq(element, other):
        new_class.add(other)
        remaining.remove(other)
    classes.append(new_class)
  return classes
def are_parallel(v1, v2):
  """
  Checks if two vectors are parallel.

  Args:
    v1: The first vector as a NumPy array.
    v2: The second vector as a NumPy array.

  Returns:
    True if the vectors are parallel, False otherwise.
  """
  v1 = np.array(v1)
  v2 = np.array(v2)
  return np.linalg.matrix_rank(np.array([v1, v2])) == 1

#equal_info = equivalence_classes(q, are_parallel)
#print(equal_info)
#equal_info
classes = equivalence_info(q,are_parallel)
classes
len(classes)


15

In [ ]:
classes

[{(0, -1, 0, 0, 1, 0, 0, 1, 0, -1, 0, 0),
  (0, 1, 0, 0, -1, 0, 0, -1, 0, 1, 0, 0)},
 {(0, 0, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0),
  (0, 0, 1, -1, 1, -1, 0, 0, 0, 0, 0, 0)},
 {(0, -1, 1, 0, 0, -1, 1, 0, 0, 0, 0, 0),
  (0, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)},
 {(0, 0, 0, -1, 0, 1, 0, 0, 0, -1, 0, 1),
  (0, 0, 0, 1, 0, -1, 0, 0, 0, 1, 0, -1)},
 {(-1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0),
  (1, 0, 0, 0, -1, -1, -1, 0, 0, 0, 0, 0)},
 {(0, 0, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0),
  (0, 0, 0, 1, 0, 0, -1, 1, -1, 0, 0, 0)},
 {(0, 0, 0, -1, 1, 0, 0, 0, -1, 1, 0, 0),
  (0, 0, 0, 1, -1, 0, 0, 0, 1, -1, 0, 0)},
 {(0, -1, 0, 0, 0, 1, 0, 0, 1, 0, -1, 0),
  (0, 1, 0, 0, 0, -1, 0, 0, -1, 0, 1, 0)},
 {(0, 0, -1, 0, 1, 0, 0, 0, 1, 0, 0, -1),
  (0, 0, 1, 0, -1, 0, 0, 0, -1, 0, 0, 1)},
 {(0, 1, 0, -1, 1, 0, -1, 0, 0, 0, 0, 0)},
 {(0, -1, 0, 0, 0, 0, 1, 0, -1, 1, 0, 0),
  (0, 1, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0)},
 {(0, 0, -1, 0, 0, 1, 0, 0, -1, 1, 0, 0),
  (0, 0, 1, 0, 0, -1, 0, 0, 1, -1, 0, 0)},
 {(-1, 1, 1, 1, 0, 0, 

pi1

In [ ]:
#classes[1]
def classrep(classes):
  classes = classes[:-1]
  numEq = len(classes)
  numBais = len(list(classes[0])[0])
  Eqs = np.zeros((numEq,numBais))
  for ind in range(numEq):
    Eqs[ind] = list(classes[ind])[0]
  #print(Eqs
  #print(numEq)
  #순차접근
  #리스트로 변환
  #norm 값이 가장 작은 애들 중에
  #1st sign이 positive인 녀석
  for ind in range(len(classes)):#ind represent ith equation
    classNow = list(classes[ind])
    print(classNow)
    classNorm = calculate_norms(classNow)
    #print(classNorm)
    setNorm = equivalence_classes(classNorm, float_equal)
    if len(setNorm) == 1:
      eqRep = np.array(sorted(classNow)[-1])
    else:
      raise "예외처리 하셈 ^^"
    Eqs[ind,:] = eqRep
  return Eqs

#return classrepArr
def calculate_norms(A):
  """
  리스트 A에 있는 각 튜플의 norm 값을 계산하여 리스트 B로 반환합니다.

  Args:
    A: 길이가 n인 리스트, 각 원소는 길이가 m인 정수 튜플입니다.

  Returns:
    길이가 n인 리스트 B, 각 원소는 A의 해당 튜플의 norm 값입니다.
  """
  B = []
  for tuple_ in A:
    vector = np.array(tuple_)  # 튜플을 NumPy 배열로 변환
    norm = np.linalg.norm(vector)  # 벡터의 norm 계산
    B.append(norm)
  return B
def float_equal(a, b, tol=0.001):
  """
  주어진 허용 오차 내에서 두 float 값이 같은지 비교합니다.

  Args:
    a: 첫 번째 float 값.
    b: 두 번째 float 값.
    tol: 허용 오차 (기본값: 1e-9).

  Returns:
    두 값이 허용 오차 내에서 같으면 True, 그렇지 않으면 False.
  """
  return np.abs(a - b) < tol
Eqs = classrep(classes)
print(Eqs)
print(Eqs.shape)

[(0, -1, 0, 0, 1, 0, 0, 1, 0, -1, 0, 0), (0, 1, 0, 0, -1, 0, 0, -1, 0, 1, 0, 0)]
[(0, 0, 1, -1, 1, -1, 0, 0, 0, 0, 0, 0), (0, 0, -1, 1, -1, 1, 0, 0, 0, 0, 0, 0)]
[(0, -1, 1, 0, 0, -1, 1, 0, 0, 0, 0, 0), (0, 1, -1, 0, 0, 1, -1, 0, 0, 0, 0, 0)]
[(0, 0, 0, 1, 0, -1, 0, 0, 0, 1, 0, -1), (0, 0, 0, -1, 0, 1, 0, 0, 0, -1, 0, 1)]
[(1, 0, 0, 0, -1, -1, -1, 0, 0, 0, 0, 0), (-1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0)]
[(0, 0, 0, -1, 0, 0, 1, -1, 1, 0, 0, 0), (0, 0, 0, 1, 0, 0, -1, 1, -1, 0, 0, 0)]
[(0, 0, 0, 1, -1, 0, 0, 0, 1, -1, 0, 0), (0, 0, 0, -1, 1, 0, 0, 0, -1, 1, 0, 0)]
[(0, 1, 0, 0, 0, -1, 0, 0, -1, 0, 1, 0), (0, -1, 0, 0, 0, 1, 0, 0, 1, 0, -1, 0)]
[(0, 0, -1, 0, 1, 0, 0, 0, 1, 0, 0, -1), (0, 0, 1, 0, -1, 0, 0, 0, -1, 0, 0, 1)]
[(0, 1, 0, -1, 1, 0, -1, 0, 0, 0, 0, 0)]
[(0, 1, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0), (0, -1, 0, 0, 0, 0, 1, 0, -1, 1, 0, 0)]
[(0, 0, 1, 0, 0, -1, 0, 0, 1, -1, 0, 0), (0, 0, -1, 0, 0, 1, 0, 0, -1, 1, 0, 0)]
[(1, -1, -1, -1, 0, 0, 0, 0, 0, 0, 0, 0), (-1, 1, 1, 1, 0, 0, 0, 0, 

In [ ]:
Eqs
Eqs = Eqs.astype(int)
print(Eqs)

[[ 0  1  0  0 -1  0  0 -1  0  1  0  0]
 [ 0  0  1 -1  1 -1  0  0  0  0  0  0]
 [ 0  1 -1  0  0  1 -1  0  0  0  0  0]
 [ 0  0  0  1  0 -1  0  0  0  1  0 -1]
 [ 1  0  0  0 -1 -1 -1  0  0  0  0  0]
 [ 0  0  0  1  0  0 -1  1 -1  0  0  0]
 [ 0  0  0  1 -1  0  0  0  1 -1  0  0]
 [ 0  1  0  0  0 -1  0  0 -1  0  1  0]
 [ 0  0  1  0 -1  0  0  0 -1  0  0  1]
 [ 0  1  0 -1  1  0 -1  0  0  0  0  0]
 [ 0  1  0  0  0  0 -1  0  1 -1  0  0]
 [ 0  0  1  0  0 -1  0  0  1 -1  0  0]
 [ 1 -1 -1 -1  0  0  0  0  0  0  0  0]
 [ 0  0  1  0  0  0 -1  0  0  1 -1  0]]


In [ ]:
foo1.columns

Index([(z[0 1], z[0 1], z[0 1]), (z[0 1], z[1 0], z[2 2]),
       (z[0 1], z[1 1], z[2 1]), (z[0 1], z[1 2], z[2 0]),
       (z[0 1], z[2 0], z[1 2]), (z[0 1], z[2 1], z[1 1]),
       (z[0 1], z[2 2], z[1 0]), (z[1 0], z[1 0], z[1 0]),
       (z[1 0], z[1 1], z[1 2]), (z[1 0], z[1 2], z[1 1]),
       (z[1 1], z[1 1], z[1 1]), (z[1 2], z[1 2], z[1 2])],
      dtype='object')

In [ ]:
D, L, R = smith_normal_form(Eqs)
print(D)
print(L)
print(R)

[[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 3 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]]
[[ 1  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0  1  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 -1 -1  0  0  0  0  0  0  0  0  0  0  0]
 [ 1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0]
 [ 1 -1 -1 -1  1  0  0  0  0  0  0  0  0  0]
 [ 1 -1 -1  0  0 -1  0  0  0  0  0  0  0  0]
 [ 1 -2 -2 -1  0 -1  0  1  0  0  0  0  0  0]
 [-2  2  2  0  0  1  1  0  0  0  0  0  0  0]
 [-1  0  1  1  0 -1  0  0  1  0  0  0  0  0]
 [ 0 -1 -1  0  0  0  0  0  0  1  0  0  0  0]
 [ 0 -1 -1  0  0  0 -1  0  0  0  1  0  0  0]
 [ 0 -1  0  0  0  0 -1  0  0  0  0  1  0  0]
 [ 1  1  0  0 -1  1  1  0  0  0  0  0  1  0]
 [-1 -1  0  0  0 -1  0  1  0  0  0  

In [ ]:
np.matmul(L,Eqs)


array([[ 0,  1,  0,  0, -1,  0,  0, -1,  0,  1,  0,  0],
       [ 0,  0,  1, -1,  1, -1,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  1, -2,  0,  1, -1,  0,  1,  0,  0],
       [ 0,  0,  0,  0, -2,  1,  1, -1,  0,  0,  0,  1],
       [ 1,  0,  0,  0, -3,  0,  0, -1,  0,  0,  0,  1],
       [ 0,  0,  0,  0, -2,  0,  2, -2,  1,  1,  0,  0],
       [ 0,  0,  0,  0, -3,  0,  3, -2,  0,  0,  1,  1],
       [ 0,  0,  0,  0,  3,  0, -3,  3,  0, -3,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0]])

pi2

In [ ]:
print(D.shape)
np.diagonal(D)

(14, 12)


array([1, 1, 1, 1, 1, 1, 1, 3, 0, 0, 0, 0])